In [18]:
#Dependencies
import pandas as pd
import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim #LDA visualization library

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

In [19]:
text_data = [
    '''I slept with this book after I read it. I kid you not: I held its bulking, hardcover bound 700 pages in my arms as I fell asleep amid a raging storm. I refused to let A Little Life leave me. Its brilliant writing, its broken characters, and its bleak, unforgiving story dug into my heart, into the very pores of my skin. As a twenty-year-old, I felt both so young and so old upon finishing this novel, as if its sheer humanity aged my soul while making me appreciate all the years I still have left.

A Little Life follows four friends after they graduate from a small, prestigious Massachusetts college: Willem, a kind and talented actor; JB, a sharp and sometimes-caustic artist, Malcolm, an aspiring architect at a well-known firm; and Jude, a mysterious and intelligent litigator. What looks like an average bildungsroman turns into an intense and tragic tale when we learn about enigmatic Jude's backstory. Abandoned at a monastery at birth, he endured a childhood of severe physical and emotional abuse, followed by several years of sexual abuse, forced prostitution, and psychological trauma. The book soon hones in on Jude's struggle to free himself from the demons of his past, the hyenas that howl and drown out the voices of his closest, most beloved friends.

This book is relentlessly sad and exquisitely written. Hanya Yanagihara spares us no mercy when revealing Jude's trauma. She details both his past abuse and his present self-harm with explicit specificity, her diction so precise and piercing it made me shake, and at times, sob. Yanagihara writes both Jude's suffering and his friendships with a keen eye. She captures the nuances of human emotion, physical space, and change over time with eloquence and heart. She writes about some of the most wretched, abominable acts of cruelty I have ever read without sentimentalizing any of the abuse or making any of the characters' feelings mawkish.

Yanagihara offers us temporary respite from the pain within Jude's past by showing us the power of friendship. A Little Life's most affective moments come not from its graphic depictions of violence, but from its quiet, uplifting portrayals of compassion. While the many abusive men in Jude's earlier life show us the depth of human atrocity, Jude's tender, bittersweet relationships with Willem, Harold, Andy, and others offer to us mankind's capacity for kindness. All of these complex characters make mistakes, and through their imperfections shines their humanness.

Please keep in mind: A Little Life is ruthlessly depressing. In the end, Jude really receives no reprieve from his anguish. As someone who has suffered his own abuse - a version less intense than Jude's, yet still real - and as someone who reads a lot about abuse, I appreciated Yanagihara's dedication to showing the darker side of reality. Trauma is trauma is trauma. And while we can all fight for recovery, sometimes that absolvement may never come. Sometimes, we just have to act with whatever kindness we have left and hope that it brings even a moment of light into the dark.

Highly recommended to anyone who wants their heart both filled and destroyed. Set aside some quality time for A Little Life. It will consume you.''',
''' It may sound presumptuous to say in January that I've read the best book I've read all year, but reading is a lot like love. Sometimes you just know.

A LITTLE LIFE is a title with 3 meanings. First, it refers to its protagonist, Jude, a man who cannot ever accept that his life is worthwhile. Second, it refers to the act of reading it, spending time in this book is really like living a version of life.

There is a third meaning, one that you don't discover until around halfway through the book when the title's words are used in a context that is like a punch to the gut. When you read them you may find yourself having a physical reaction, your stomach may flip, your skin may go cold, you may gasp for breath. And that is a lot of what the experience of reading this book is like. You can call these parts of the book words like "awful," but to be real you'd need to pull out your thesaurus and just line them up one after the other. This is not a book that is easy for your emotions. You care about the people in it, so the pain can really hurt you.

You will hear that this is a book about 4 friends. It's not. They're a nice framing device, but this is a book about one person and the people who are connected to him. His life is made up of extremes. I found myself weeping over and over again because of the love and compassion and kindness that characters in the book displayed. But this book has some of the most harrowing and horrifying scenes I've read anywhere. It is not really spoiling anything to say this involves terrible things happening to a child. Everyone knows from the very beginning that something bad happened to Jude when he was young. It's just so much worse than you could imagine. (If you have trouble reading about child abuse, it's probably best you not read this book. While it's essential to the story, it is not glossed over or referenced vaguely and what is described is truly terrible to contemplate.)

Jude is not a new character. The damaged soul whose self-worth never really recovers is present in a lot of modern fiction. Yanagihara's trick, I think, is just how willing she is to plumb the depths of his darkness and its effects on those around him. She follows him for decades, observes him in all situations, and is unflinching in her depictions. Her writing is the kind of good that you can miss if you're not paying attention. You are so caught up in her story that it's easy to miss just how agile and careful the book is. It eases back and forth from character to character, backwards and forwards in time, and it never feels strained.

I stayed up for hours to finish this book and then couldn't sleep because I couldn't let it go. I was overcome by the book and by the loss of finishing it.

This is a book about love and what it means and what it can do and it is the humanity of its characters and their love for each other that will stick with me.

If it was presumptuous to say this is my favorite book of 2015 since it tries to predict the future, I do feel that I can dig through the past and assert with certainty that this is one of the best books I've ever read.''',
'''Around page 150 of Hanya Yanagihara’s second novel, A Little Life, which follows four friends from their college years into their fifties, I wrote the following in my notes:

I am more excited about Hanya Yanagihara and her work and her career than I have been about any author in a really long time.

Around page 200 I wrote this:

Is Jude’s suffering perhaps a tad overwrought? It is starting to seem like everything bad happens to him forever. Maybe we should spend some time with one of the other characters.

Page 200 Jenny was right, and Page 150 Jenny was — well, hope springs eternal, and maybe Yanagihara’s third book will be back up to the standard of The People in the Trees. But as for A Little Life, describing Jude’s suffering as “a tad overwrought” is like describing Dolores Umbridge as “a tad unpleasant.” Yanagihara employs a plot strategy of which I was very fond when I was eleven, which was to think of as many dreadful fates as I could and heap them upon my protagonist one after another. Then when I ran out of ideas, I killed the protagonist off and wrote heartrending scenes of her friends-and-relations mourning her wretched life and too-early passing. I did this because I was eleven. I am not sure what Yanagihara’s problem is.

We learn early on that Jude is physically frail, due to an unspecified injury in his past, and that his family isn’t in the picture. Over the course of seven hundred pages, Yanagihara unfolds a cartoonishly woeful backstory to explain all of this. When you first start to recognize the way Jude’s abusive past is tearing him apart in the present, it’s heartbreaking. After two or three wicked villains have gotten through abusing him just because they’re evil, you start worrying that if the author doesn’t right the ship, you’re going to find yourself in the unenviable position of describing a depiction of child sex abuse as silly in your eventual review.

The maddening waste is that Yanagihara’s writing is elegant and evocative, and she’s able — at times — to capture with precision and delicacy the true, messy emotions between her characters. And the kind of story that she’s (I think) trying to tell is a kind of story I want to see more of. I want a story that doesn’t pretend there’s a straight path out of trauma into healing that you travel once and then you reach the end and you and your trauma have no further business to transact. I want a story that places serious value on relationships other than romantic ones. I want a story about loving someone who cannot always see his way clear to continuing to live in this world.

Ideally, of course, these stories would reach me unencumbered by several metric tons of lunatic melodrama, and I would not have to use the word dreck in reference to an author I admire. But in this I am evidently destined for disappointment.'''    
   
]

In [20]:
import pandas as pd

# Create a DataFrame from the text data
df = pd.DataFrame({
    'text': text_data
})

# Print the shape and contents of the DataFrame
print(df.shape)
print(df)

(3, 1)
                                                text
0  I slept with this book after I read it. I kid ...
1   It may sound presumptuous to say in January t...
2  Around page 150 of Hanya Yanagihara’s second n...


In [21]:
#clean the data
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [22]:
df['text_clean']=df['text'].apply(clean)
df

,text,text_clean
0,I slept with this book after I read it. I kid ...,"[slept, book, read, it, kid, not, held, bulkin..."
1,It may sound presumptuous to say in January t...,"[may, sound, presumptuous, say, january, ive, ..."
2,Around page 150 of Hanya Yanagihara’s second n...,"[around, page, 150, hanya, yanagihara’s, secon..."


In [25]:
#create dictionary
dictionary = corpora.Dictionary(df['text_clean'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)

637


In [26]:
#create document term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['text_clean'] ]
print(len(doc_term_matrix))

3


In [27]:
lda = gensim.models.ldamodel.LdaModel

In [28]:
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [29]:
def evaluate_lda_model(corpus, dictionary, num_topics_list, texts):
    coherence_scores = {}
    perplexity_scores = {}

    for num_topics in num_topics_list:
        # Train the LDA model
        lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=50, minimum_probability=0)
        
        # Compute Coherence Score
        coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_scores[num_topics] = coherence_model.get_coherence()
        
        # Compute Perplexity Score
        perplexity_scores[num_topics] = lda_model.log_perplexity(corpus)

    # Select the best number of topics based on coherence score
    best_num_topics_coherence = max(coherence_scores, key=coherence_scores.get)
    # Select the best number of topics based on perplexity score
    best_num_topics_perplexity = min(perplexity_scores, key=perplexity_scores.get)

    # You can choose based on coherence, perplexity, or a combination
    best_num_topics = best_num_topics_coherence  # or best_num_topics_perplexity

    return best_num_topics, coherence_scores, perplexity_scores

In [16]:
num_topics_list = [5, 10, 15, 20]  # List of different numbers of topics to evaluate
best_num_topics, coherence_scores, perplexity_scores = evaluate_lda_model(doc_term_matrix, dictionary, num_topics_list, df['text_clean'])

print(f'Best number of topics based on coherence: {best_num_topics}')
print('Coherence Scores:', coherence_scores)
print('Perplexity Scores:', perplexity_scores)

Best number of topics based on coherence: 20
Coherence Scores: {5: 0.7482149915498593, 10: 0.7272448102281477, 15: 0.647122539456005, 20: 0.8081543989915103}
Perplexity Scores: {5: -5.968697388526992, 10: -6.071455259238076, 15: -6.2190478051134, 20: -6.100038024992081}


In [32]:
num_topics=2

ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.012*"story" + 0.010*"page" + 0.008*"want" + 0.008*"yanagihara’s" + 0.007*"yanagihara" + 0.007*"one" + 0.007*"time" + 0.007*"describing" + 0.007*"wrote" + 0.007*"tad"'),
 (1,
  '0.024*"book" + 0.016*"jude" + 0.012*"life" + 0.011*"read" + 0.010*"character" + 0.009*"abuse" + 0.008*"little" + 0.008*"like" + 0.008*"may" + 0.007*"really"')]